# ROC-Kurven einfach erklärt

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_12/roc_kurve_demo.ipynb)

**Lernziel:** Verstehen, wie ROC-Kurven entstehen und interpretiert werden.

**Beispiel:** Defekterkennung in der Produktion - 100 Bauteile werden getestet.

Wir nutzen hier erstellen und nutzen hier kein Modell, sondern simulieren einen Datensatz. Es geht um die Erklärung der sogenannten "ROC-Kurve".

**Herkunft und Bedeutung**

Der Begriff stammt nicht ursprünglich aus der Statistik oder dem maschinellen Lernen, sondern aus der Radar- und Signalverarbeitung während des Zweiten Weltkriegs:

„Receiver“ = Empfänger (Radar-Empfangsgerät)

„Operating Characteristic“ = beschreibt, wie dieser Empfänger unter verschiedenen Schwellwert-Einstellungen arbeitet

Damals wollte man herausfinden, wie gut ein Radar zwischen „Signal vorhanden“ (z. B. Flugzeug erkannt) und „kein Signal“ (Rauschen) unterscheiden kann.
Dazu variierte man den Entscheidungsschwellenwert: je niedriger er war, desto mehr Treffer, aber auch mehr Fehlalarme; je höher, desto weniger Fehlalarme, aber auch mehr verpasste Signale.
Die ROC-Kurve zeigte diesen Trade-off zwischen Trefferquote (True Positive Rate) und Falschalarmrate (False Positive Rate).

**Übertragung auf maschinelles Lernen**

In der Statistik und im Machine Learning nutzt man die ROC-Kurve heute genau für denselben Zweck:

Sie zeigt, wie gut ein Klassifikationsmodell bei verschiedenen Schwellwerten zwischen zwei Klassen unterscheiden kann.

Die Achsen sind:

x-Achse: False Positive Rate (1 − Spezifität)

y-Achse: True Positive Rate (Sensitivität)

Der Flächeninhalt unter der ROC-Kurve (AUC) dient als Maß für die Trennschärfe des Modells.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("✓ Bibliotheken geladen")

## 📊 Unser Beispiel-Datensatz

**100 Bauteile getestet:**
- 90 sind tatsächlich **OK** (gehören zur "negativen Kategorie", Defektwahrscheinlichkeit klein)
- 10 sind tatsächlich **NOK/defekt** (gehören zur "positiven Kategorie", Defektwahrscheinlichkeit hoch))

**Unser Modell gibt (Defekt-)Wahrscheinlichkeiten zwischen 0 und 1 aus:**
- Hohe Defekt-Wahrscheinlichkeit → "Wahrscheinlich defekt"
- Niedrige Defekt-Wahrscheinlichkeit → "Wahrscheinlich OK"

**Die "OK-Wahrscheinlichkeit" eines Teils ist dann 1 minus die Defektwahrscheinlichkeit:**

Beispiel: ein Teil ist mit 70% Wahrscheinlichkeit defekt --> mit 30% Wahrscheinlichkeit in Ordnung

**Die finale Entscheidung "defekt oder nicht" wird mit einem Schwellwert getroffen:**

z.B. Schwellwert bei 50%: wenn Defektwahrscheinlichkeit > 50% --> Teil defekt!

In [ ]:
# Realistisches Beispiel mit Fehlklassifikationen
np.random.seed(42)  # Für reproduzierbare Ergebnisse

# Wahre Labels: 90x OK (0), 10x defekt (1)
wahre_labels = [0] * 90 + [1] * 10

# Realistische Wahrscheinlichkeiten mit Überlappungen
# OK-Teile: Normalverteilung um 0.3 (manche haben trotzdem höhere Werte!)
ok_wahrscheinlichkeiten = np.random.normal(0.3, 0.15, 90)
ok_wahrscheinlichkeiten = np.clip(ok_wahrscheinlichkeiten, 0.05, 0.95)  # Zwischen 0.05 und 0.95

# Defekte Teile: Normalverteilung um 0.7 (manche haben trotzdem niedrigere Werte!)
defekt_wahrscheinlichkeiten = np.random.normal(0.7, 0.15, 10)
defekt_wahrscheinlichkeiten = np.clip(defekt_wahrscheinlichkeiten, 0.05, 0.95)  # Zwischen 0.05 und 0.95

# Alle Wahrscheinlichkeiten zusammen
wahrscheinlichkeiten = np.concatenate([ok_wahrscheinlichkeiten, defekt_wahrscheinlichkeiten])

print(f"📊 REALISTISCHER DATENSATZ:")
print(f"Total: {len(wahre_labels)} Bauteile")
print(f"OK-Teile: {wahre_labels.count(0)}")
print(f"Defekte Teile: {wahre_labels.count(1)}")
print(f"\nBeispiel-Wahrscheinlichkeiten:")
print(f"OK-Teile: {ok_wahrscheinlichkeiten[:5].round(3)} ...")
print(f"Defekte Teile: {defekt_wahrscheinlichkeiten.round(3)}")

print(f"\n🔍 WICHTIG: Überlappungen vorhanden!")
print(f"Höchste OK-Wahrscheinlichkeit: {ok_wahrscheinlichkeiten.max():.3f}")
print(f"Niedrigste Defekt-Wahrscheinlichkeit: {defekt_wahrscheinlichkeiten.min():.3f}")
print(f"→ Perfekte Trennung ist unmöglich!")

# Zeige problematische Fälle
hohe_ok = ok_wahrscheinlichkeiten[ok_wahrscheinlichkeiten > 0.5]
niedrige_defekt = defekt_wahrscheinlichkeiten[defekt_wahrscheinlichkeiten < 0.5]

print(f"\n⚠️ PROBLEMATISCHE FÄLLE:")
print(f"OK-Teile mit hoher Defekt-Wahrscheinlichkeit (>0.5, 'Pseudofehler', 'false positive'): {len(hohe_ok)} Stück")
print(f"Defekte Teile mit niedriger Defekt-Wahrscheinlichkeit (<0.5, 'Schlupf', 'false negative'): {len(niedrige_defekt)} Stück")
print(f"→ Diese werden bei Schwellwert 0.5 falsch klassifiziert!")

## 🎯 Was ist der Schwellwert?

**Der Schwellwert entscheidet die Klassifikation:**
- Wahrscheinlichkeit ≥ Schwellwert → "Defekt" vorhersagen
- Wahrscheinlichkeit < Schwellwert → "OK" vorhersagen

**Problem:** Die Wahrscheinlichkeiten überlappen sich!
- Manche OK-Teile haben hohe Wahrscheinlichkeiten (Fehlalarm)
- Manche defekte Teile haben niedrige Wahrscheinlichkeiten (übersehen)

**Beispiel bei Schwellwert 0.5:**
- OK-Teil mit 0.6 → fälschlich als "Defekt" klassifiziert (FP)
- Defektes Teil mit 0.4 → fälschlich als "OK" klassifiziert (FN)

In [ ]:
def berechne_metriken(wahre_labels, wahrscheinlichkeiten, schwellwert):
    """Berechnet TPR und FPR für einen gegebenen Schwellwert"""
    
    # Vorhersagen basierend auf Schwellwert
    vorhersagen = [1 if prob >= schwellwert else 0 for prob in wahrscheinlichkeiten]
    
    # Konfusionsmatrix berechnen
    TP = sum(1 for true, pred in zip(wahre_labels, vorhersagen) if true == 1 and pred == 1)
    TN = sum(1 for true, pred in zip(wahre_labels, vorhersagen) if true == 0 and pred == 0)
    FP = sum(1 for true, pred in zip(wahre_labels, vorhersagen) if true == 0 and pred == 1)
    FN = sum(1 for true, pred in zip(wahre_labels, vorhersagen) if true == 1 and pred == 0)
    
    # TPR und FPR berechnen
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
    
    return TPR, FPR, TP, TN, FP, FN

# Beispiel für Schwellwert 0.5
schwellwert_beispiel = 0.5
tpr, fpr, tp, tn, fp, fn = berechne_metriken(wahre_labels, wahrscheinlichkeiten, schwellwert_beispiel)

print(f"📐 BEISPIEL: Schwellwert = {schwellwert_beispiel}")
print(f"="*40)
print(f"TP (richtig als defekt erkannt): {tp}")
print(f"TN (richtig als OK erkannt): {tn}")
print(f"FP (OK fälschlich als defekt): {fp}")
print(f"FN (defekt fälschlich als OK): {fn}")
print(f"")
print(f"TPR = {tp}/({tp}+{fn}) = {tpr:.3f} = {tpr*100:.1f}%")
print(f"FPR = {fp}/({fp}+{tn}) = {fpr:.3f} = {fpr*100:.1f}%")
print(f"")
print(f"Interpretation:")
print(f"• {tpr*100:.1f}% aller defekten Teile werden erkannt")
print(f"• {fpr*100:.1f}% aller OK-Teile werden fälschlich als defekt eingestuft")

## 📈 ROC-Kurve: Verschiedene Schwellwerte testen

**Idee der ROC-Kurve:**
1. Teste viele verschiedene Schwellwerte (0.1, 0.2, 0.3, ..., 0.9)
2. Berechne für jeden Schwellwert TPR und FPR
3. Trage jeden Punkt (FPR, TPR) in ein Diagramm ein
4. Verbinde die Punkte → ROC-Kurve!

In [ ]:
# Verschiedene Schwellwerte testen
schwellwerte = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
tpr_werte = []
fpr_werte = []

print("📈 ROC-KURVE SCHRITT FÜR SCHRITT:")
print("="*50)
print("Schwellwert | TPR   | FPR   | Was bedeutet das?")
print("-"*50)

for schwellwert in schwellwerte:
    tpr, fpr, tp, tn, fp, fn = berechne_metriken(wahre_labels, wahrscheinlichkeiten, schwellwert)
    
    tpr_werte.append(tpr)
    fpr_werte.append(fpr)
    
    # Interpretation
    if schwellwert <= 0.3:
        bedeutung = "Sehr sensitiv - wenig übersehen"
    elif schwellwert <= 0.6:
        bedeutung = "Ausgewogen"
    else:
        bedeutung = "Sehr spezifisch - wenig Fehlalarme"
    
    print(f"   {schwellwert:.1f}      | {tpr:.3f} | {fpr:.3f} | {bedeutung}")

print(f"\n🎯 BEOBACHTUNG:")
print(f"• Niedriger Schwellwert → Hohe TPR, aber auch hohe FPR")
print(f"• Hoher Schwellwert → Niedrige FPR, aber auch niedrige TPR")
print(f"• Das ist der Trade-off!")

In [ ]:
# ROC-Kurve visualisieren - Teil 1: Grundlegende Plots
plt.figure(figsize=(12, 8))

# FPR und TPR Werte für korrekte AUC-Berechnung sortieren
fpr_tpr_pairs = list(zip(fpr_werte, tpr_werte, schwellwerte))
fpr_tpr_pairs.sort()  # Sortiert nach FPR
fpr_sortiert = [pair[0] for pair in fpr_tpr_pairs]
tpr_sortiert = [pair[1] for pair in fpr_tpr_pairs]

# Subplot 1: ROC-Kurve
plt.subplot(2, 2, 1)
plt.plot(fpr_sortiert, tpr_sortiert, 'bo-', linewidth=2, markersize=8, label='Unser Modell')
plt.plot([0, 1], [0, 1], 'r--', linewidth=1, alpha=0.7, label='Zufall')

# Einige Schwellwerte beschriften
for i, (fpr, tpr, schwellwert) in enumerate(zip(fpr_werte, tpr_werte, schwellwerte)):
    if i % 3 == 0:  # Nur jeden dritten Punkt beschriften
        plt.annotate(f'{schwellwert:.1f}', (fpr, tpr), 
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.xlabel('FPR (False Positive Rate)')
plt.ylabel('TPR (True Positive Rate)')
plt.title('ROC-Kurve')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: TPR und TNR vs Schwellwert
plt.subplot(2, 2, 2)
plt.plot(schwellwerte, tpr_werte, 'go-', linewidth=2, label='TPR (Sensitivität)')
plt.plot(schwellwerte, [1-fpr for fpr in fpr_werte], 'ro-', linewidth=2, label='TNR (Spezifität)')
plt.xlabel('Schwellwert')
plt.ylabel('Rate')
plt.title('TPR und TNR vs Schwellwert')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# AUC berechnen
auc_schaetzung = np.trapezoid(tpr_sortiert, fpr_sortiert)
print(f"🎯 ROC-KURVE ERSTELLT:")
print(f"• AUC = {auc_schaetzung:.3f} (Fläche unter der Kurve)")
print(f"• AUC > 0.5 = besser als Zufall ✓")
print(f"• Je näher an 1.0, desto besser das Modell")

In [ ]:
# Wahrscheinlichkeits-Verteilungen visualisieren
plt.figure(figsize=(10, 6))

plt.hist(ok_wahrscheinlichkeiten, bins=15, alpha=0.7, label='OK-Teile (90 Stück)', color='green')
plt.hist(defekt_wahrscheinlichkeiten, bins=15, alpha=0.7, label='Defekte Teile (10 Stück)', color='red')
plt.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Schwellwert 0.5')

# Fehlklassifikations-Bereiche markieren
plt.axvspan(0.5, 1.0, alpha=0.2, color='red')
plt.axvspan(0.0, 0.5, alpha=0.2, color='orange')

plt.xlabel('Wahrscheinlichkeit "defekt"')
plt.ylabel('Anzahl Bauteile')
plt.title('Wahrscheinlichkeits-Verteilungen mit Überlappung')
plt.legend()

# Erklärende Texte
plt.text(0.25, plt.ylim()[1]*0.7, 'FN-Bereich:\nDefekte als OK', ha='center', fontsize=10,
         bbox=dict(boxstyle="round", facecolor="orange", alpha=0.8))
plt.text(0.75, plt.ylim()[1]*0.7, 'FP-Bereich:\nOK als defekt', ha='center', fontsize=10,
         bbox=dict(boxstyle="round", facecolor="red", alpha=0.8))

plt.tight_layout()
plt.show()

print(f"🔍 ÜBERLAPPUNGS-PROBLEM:")
print(f"• Grüne Balken rechts von 0.5 = Fehlalarme (FP)")
print(f"• Rote Balken links von 0.5 = Übersehene Defekte (FN)")
print(f"• Überlappung macht perfekte Klassifikation unmöglich")
print(f"• Deshalb Trade-off zwischen TPR und FPR notwendig")

## 🎯 Optimalen Schwellwert wählen

**Die Frage:** Welcher Punkt auf der ROC-Kurve ist der beste?

**Die Antwort:** Hängt vom Anwendungsfall ab!

### 📝 Was bedeuten die Strategien?

**🚨 Vorsichtig (niedriger Schwellwert):**
- System sagt schnell "Das ist defekt!" 
- Schon bei geringer Wahrscheinlichkeit wird klassifiziert
- ➤ Wenig wird übersehen, aber viele Fehlalarme

**⚖️ Standard (mittlerer Schwellwert):**
- System entscheidet bei 50% Wahrscheinlichkeit
- Ausgewogener Kompromiss

**🛡️ Zurückhaltend (hoher Schwellwert):**
- System sagt erst bei hoher Sicherheit "Das ist defekt!"
- Nur bei hoher Wahrscheinlichkeit wird klassifiziert  
- ➤ Wenige Fehlalarme, aber mehr wird übersehen

### 🎯 Anwendungsbeispiele:
- **Qualitätskontrolle:** Lieber vorsichtig → niedrigerer Schwellwert
- **Spam-Filter:** Lieber zurückhaltend → höherer Schwellwert

In [ ]:
# Drei verschiedene Strategien vergleichen
strategien = {
    'Vorsichtig (0.3)': 0.3,  # Niedrig = sagt schnell "defekt!"
    'Standard (0.5)': 0.5,    # Ausgewogen = 50% Grenze
    'Zurückhaltend (0.7)': 0.7   # Hoch = braucht hohe Sicherheit
}

print("🎯 SCHWELLWERT-STRATEGIEN VERGLEICH:")
print("="*60)

for name, schwellwert in strategien.items():
    tpr, fpr, tp, tn, fp, fn = berechne_metriken(wahre_labels, wahrscheinlichkeiten, schwellwert)
    
    print(f"\n📊 {name.upper()}:")
    print(f"   Schwellwert: {schwellwert}")
    print(f"   TPR: {tpr:.1%} → {tp} von {tp+fn} defekten Teilen erkannt")
    print(f"   FPR: {fpr:.1%} → {fp} von {fp+tn} OK-Teilen als Fehlalarm")
    print(f"   Übersehene defekte Teile: {fn}")
    print(f"   Verschwendete OK-Teile: {fp}")
    
    # Bewertung mit Erklärung
    if schwellwert == 0.3:
        print(f"   ➤ VORSICHTIG: Sagt schnell 'defekt!' → wenig übersehen, viele Fehlalarme")
    elif schwellwert == 0.5:
        print(f"   ➤ STANDARD: Ausgewogener Kompromiss bei 50%")
    else:
        print(f"   ➤ ZURÜCKHALTEND: Braucht hohe Sicherheit → wenig Fehlalarme, mehr übersehen")

# Visualisierung der Strategien
plt.figure(figsize=(10, 6))

# ROC-Kurve mit markierten Strategien
plt.plot(fpr_werte, tpr_werte, 'b-', linewidth=2, alpha=0.7, label='ROC-Kurve')
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Zufall')

# Strategien als Punkte markieren
farben = ['red', 'orange', 'green']
for i, (name, schwellwert) in enumerate(strategien.items()):
    # Finde den entsprechenden Punkt
    idx = schwellwerte.index(schwellwert)
    plt.plot(fpr_werte[idx], tpr_werte[idx], 'o', 
            color=farben[i], markersize=12, label=name)
    
    # Beschriftung
    plt.annotate(name.split('(')[0], 
                (fpr_werte[idx], tpr_werte[idx]),
                xytext=(10, 10), textcoords='offset points', 
                fontsize=10, fontweight='bold')

plt.xlabel('FPR (False Positive Rate)')
plt.ylabel('TPR (True Positive Rate)')
plt.title('ROC-Kurve mit verschiedenen Strategien')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n💡 FAZIT:")
print(f"• ROC-Kurve zeigt alle möglichen Trade-offs")
print(f"• Optimaler Punkt hängt von Geschäftszielen ab")
print(f"• Qualitätskontrolle: Lieber vorsichtig (hohe TPR)")
print(f"• Kostenkontrolle: Lieber zurückhaltend (niedrige FPR), aber Achtung: false negatives können später viel teurer werden!")

## 📚 Zusammenfassung: ROC-Kurven verstehen

**Was wir gelernt haben:**

1. **Schwellwerte** entscheiden, ab welcher Wahrscheinlichkeit wir "positiv" vorhersagen

2. **ROC-Kurve entsteht** durch Variation des Schwellwerts:
   - Jeder Schwellwert → ein Punkt (FPR, TPR)
   - Alle Punkte verbinden → ROC-Kurve

3. **Interpretation:**
   - Links oben = gut (hohe TPR, niedrige FPR)
   - Diagonale = Zufall (nutzlos)
   - AUC = Fläche unter Kurve (>0.5 = besser als Zufall)

4. **Optimaler Schwellwert** hängt vom Anwendungsfall ab:
   - Medizin/Qualität: Hohe TPR wichtig
   - Spam/Kosten: Niedrige FPR wichtig